<a href="https://colab.research.google.com/github/Janina712/MLTSA22_JBoecher/blob/main/Other/BreathGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. Imports & Set-Up**

In [1]:
import pandas as pd
import numpy as np 
import os

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [97]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/1.CombinedInputFiles


In [98]:
read_TextGrid_duration = pd.read_excel("read_TextGrid_comb.xlsx")
df_read_txt_wide = pd.read_excel("read_txt_comb.xlsx")

**1. Create empty dataframe row to store BG assignment**

In [99]:
BG_col = pd.DataFrame()

**2. Create variables needed inside loop**

In [100]:
IDs = ['24fa', '24ma', '24mb', '26f', '29ma', '29mb', '29mc', '32m', '34m', '35mb', '46ma', '50fa', '50fb', '54f', '57m', '60m', '62f', '62m', 'C10_KS', 'C11_NP', 'C12_CC', 'C13_RG','C1_DS', 'C2_JH', 'C3_MD', 'C4_ZO', 'C5_HL', 'C6_LH', 'C7_SH', 'C9_RD']

In [101]:
IntervalTimes = df_read_txt_wide[['ID', 'Offset']] 

In [102]:
Offsets_all = read_TextGrid_duration[['ID', 'Offset']]

In [103]:
ID_freq_sound = read_TextGrid_duration['ID'].value_counts().sort_index()

**3. Assign Breath Groups for All Participants in a Loop**

In [104]:
n = -1

In [105]:
for ID in IDs:
  n = n + 1
  BG_current = pd.DataFrame(index=range(int(ID_freq_sound[n])),columns=range(1))
  subset_IT = IntervalTimes[IntervalTimes["ID"] == ID]
  subset_IT.index = range(len(subset_IT.index))
  subset_sounds = Offsets_all[Offsets_all["ID"] == ID]
  subset_sounds.index = range(len(subset_sounds.index))
  for j in range(0,len(subset_IT)): # index over range of utterances from participant
    for i in range(0,len(subset_sounds)-1): # index over sounds from participant
      if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
        if pd.isna(BG_current[0][i]) == True:
          BG_current[0][i] = j+1 # assign number of breath group to corresponding row
  BG_col = BG_col.append([BG_current], ignore_index=True)


In [106]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, BG_col], axis=1)

In [108]:
read_TextGrid_duration.rename(columns = {0:'Breath.Group'}, inplace = True)

In [109]:
read_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [110]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration,Breath.Group
0,24fa,silence,"""sp""",0.000,0.013,0.013,1
1,24fa,silence,"""sp""",0.013,0.143,0.130,1
2,24fa,consonant,"""S""",0.143,0.263,0.120,1
3,24fa,vowel,"""OW1""",0.263,0.343,0.080,1
4,24fa,consonant,"""HH""",0.343,0.482,0.139,1
...,...,...,...,...,...,...,...
24338,C13_RG,consonant,"""N""",87.881,87.910,0.029,28
24339,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,28
24340,C13_RG,consonant,"""M""",88.081,88.191,0.110,28
24341,C13_RG,consonant,"""Z""",88.191,88.281,0.090,28


**4. Save Results Created by Loop**

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

In [ ]:
dir = "2.BreathGroups_Assigned"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

In [ ]:
read_TextGrid_duration.to_excel("reading_TextGrid_comb_BG_loop.xlsx")

In [ ]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

**5. Assign Breath Groups for one Participant at a Time**

In [10]:
BG_24fa = pd.DataFrame(index=range(int(ID_freq_sound[0])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24fa"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24fa"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24fa[0][i]) == True:
        BG_24fa[0][i] = j+1

In [11]:
BG_24ma = pd.DataFrame(index=range(int(ID_freq_sound[1])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24ma[0][i]) == True:
        BG_24ma[0][i] = j+1

In [12]:
BG_24mb = pd.DataFrame(index=range(int(ID_freq_sound[2])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "24mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "24mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_24mb[0][i]) == True:
        BG_24mb[0][i] = j+1

In [13]:
BG_26f = pd.DataFrame(index=range(int(ID_freq_sound[3])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "26f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "26f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_26f[0][i]) == True:
        BG_26f[0][i] = j+1

In [14]:
BG_29ma = pd.DataFrame(index=range(int(ID_freq_sound[4])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29ma[0][i]) == True:
        BG_29ma[0][i] = j+1

In [15]:
BG_29mb = pd.DataFrame(index=range(int(ID_freq_sound[5])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29mb[0][i]) == True:
        BG_29mb[0][i] = j+1

In [16]:
BG_29mc = pd.DataFrame(index=range(int(ID_freq_sound[6])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "29mc"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "29mc"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_29mc[0][i]) == True:
        BG_29mc[0][i] = j+1

In [17]:
BG_32m = pd.DataFrame(index=range(int(ID_freq_sound[7])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "32m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "32m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_32m[0][i]) == True:
        BG_32m[0][i] = j+1

In [18]:
BG_34m = pd.DataFrame(index=range(int(ID_freq_sound[8])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "34m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "34m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_34m[0][i]) == True:
        BG_34m[0][i] = j+1

In [19]:
BG_35mb = pd.DataFrame(index=range(int(ID_freq_sound[9])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "35mb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "35mb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_35mb[0][i]) == True:
        BG_35mb[0][i] = j+1

In [20]:
BG_46ma = pd.DataFrame(index=range(int(ID_freq_sound[10])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "46ma"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "46ma"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_46ma[0][i]) == True:
        BG_46ma[0][i] = j+1

In [21]:
BG_50fa = pd.DataFrame(index=range(int(ID_freq_sound[11])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fa"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "50fa"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_50fa[0][i]) == True:
        BG_50fa[0][i] = j+1

In [22]:
BG_50fb = pd.DataFrame(index=range(int(ID_freq_sound[12])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "50fb"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "50fb"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_50fb[0][i]) == True:
        BG_50fb[0][i] = j+1

In [23]:
BG_54f = pd.DataFrame(index=range(int(ID_freq_sound[13])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "54f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "54f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_54f[0][i]) == True:
        BG_54f[0][i] = j+1

In [24]:
BG_57m = pd.DataFrame(index=range(int(ID_freq_sound[14])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "57m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "57m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_57m[0][i]) == True:
        BG_57m[0][i] = j+1

In [25]:
BG_60m = pd.DataFrame(index=range(int(ID_freq_sound[15])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "60m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "60m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_60m[0][i]) == True:
        BG_60m[0][i] = j+1

In [26]:
BG_62f = pd.DataFrame(index=range(int(ID_freq_sound[16])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62f"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "62f"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_62f[0][i]) == True:
        BG_62f[0][i] = j+1

In [27]:
BG_62m = pd.DataFrame(index=range(int(ID_freq_sound[17])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "62m"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "62m"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_62m[0][i]) == True:
        BG_62m[0][i] = j+1

In [28]:
BG_C10_KS = pd.DataFrame(index=range((int(ID_freq_sound[18]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C10_KS"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C10_KS"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C10_KS[0][i]) == True:
        BG_C10_KS[0][i] = j+1

In [29]:
BG_C11_NP = pd.DataFrame(index=range(int(ID_freq_sound[19])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C11_NP"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C11_NP"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C11_NP[0][i]) == True:
        BG_C11_NP[0][i] = j+1

In [30]:
BG_C12_CC = pd.DataFrame(index=range(int(ID_freq_sound[20])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C12_CC"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C12_CC"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C12_CC[0][i]) == True:
        BG_C12_CC[0][i] = j+1

In [31]:
BG_C13_RG = pd.DataFrame(index=range(int(ID_freq_sound[21])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C13_RG"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C13_RG"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C13_RG[0][i]) == True:
        BG_C13_RG[0][i] = j+1

In [32]:
BG_C1_DS = pd.DataFrame(index=range(int(ID_freq_sound[22])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C1_DS"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C1_DS"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C1_DS[0][i]) == True:
        BG_C1_DS[0][i] = j+1

In [33]:
BG_C2_JH = pd.DataFrame(index=range(int(ID_freq_sound[23])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C2_JH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C2_JH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C2_JH[0][i]) == True:
        BG_C2_JH[0][i] = j+1

In [34]:
BG_C3_MD = pd.DataFrame(index=range((int(ID_freq_sound[24]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C3_MD"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C3_MD"]
subset_sounds.index = range(len(subset_sounds.index))
BG_C3_MD = pd.DataFrame(index=range(len(subset_sounds.index)),columns=range(1))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C3_MD[0][i]) == True:
        BG_C3_MD[0][i] = j+1

In [35]:
BG_C4_ZO = pd.DataFrame(index=range((int(ID_freq_sound[25]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C4_ZO"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C4_ZO"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C4_ZO[0][i]) == True:
        BG_C4_ZO[0][i] = j+1

In [36]:
BG_C5_HL = pd.DataFrame(index=range((int(ID_freq_sound[26]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C5_HL"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C5_HL"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C5_HL[0][i]) == True:
        BG_C5_HL[0][i] = j+1

In [37]:
BG_C6_LH = pd.DataFrame(index=range((int(ID_freq_sound[27]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C6_LH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C6_LH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C6_LH[0][i]) == True:
        BG_C6_LH[0][i] = j+1

In [38]:
BG_C7_SH = pd.DataFrame(index=range(int(ID_freq_sound[28])),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C7_SH"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C7_SH"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C7_SH[0][i]) == True:
        BG_C7_SH[0][i] = j+1

In [39]:
BG_C9_RD = pd.DataFrame(index=range((int(ID_freq_sound[29]))),columns=range(1))
subset_IT = IntervalTimes[IntervalTimes["ID"] == "C9_RD"]
subset_IT.index = range(len(subset_IT.index))
subset_sounds = Offsets_all[Offsets_all["ID"] == "C9_RD"]
subset_sounds.index = range(len(subset_sounds.index))
for j in range(0,len(subset_IT)):
  for i in range(0,len(subset_sounds)-1):
    if subset_sounds["Offset"][i] <= subset_IT["Offset"][j]:
      if pd.isna(BG_C9_RD[0][i]) == True:
        BG_C9_RD[0][i] = j+1

In [40]:
all = pd.DataFrame()

In [41]:
all = all.append([BG_24fa, BG_24ma, BG_24mb, BG_26f,BG_29ma, BG_29mb, BG_29mc, BG_32m, BG_34m, BG_35mb, BG_46ma, BG_50fa, BG_50fb, BG_54f, BG_57m, BG_60m, BG_62f, BG_62m, BG_C1_DS, BG_C2_JH, BG_C3_MD, BG_C4_ZO, BG_C5_HL, BG_C6_LH, BG_C7_SH, BG_C9_RD, BG_C10_KS, BG_C11_NP, BG_C12_CC, BG_C13_RG], ignore_index=True)


In [42]:
read_TextGrid_duration = pd.concat([read_TextGrid_duration, all], axis=1)

In [43]:
read_TextGrid_duration.rename(columns = {0:'Breath.Group'}, inplace = True)

In [44]:
read_TextGrid_duration.drop(['Unnamed: 0'], axis=1 , inplace = True)

In [45]:
read_TextGrid_duration

,ID,Type,Sound,Onset,Offset,Duration,Breath.Group
0,24fa,silence,"""sp""",0.000,0.013,0.013,1
1,24fa,silence,"""sp""",0.013,0.143,0.130,1
2,24fa,consonant,"""S""",0.143,0.263,0.120,1
3,24fa,vowel,"""OW1""",0.263,0.343,0.080,1
4,24fa,consonant,"""HH""",0.343,0.482,0.139,1
...,...,...,...,...,...,...,...
24338,C13_RG,consonant,"""N""",87.881,87.910,0.029,33
24339,C13_RG,vowel,"""EY2""",87.910,88.081,0.171,33
24340,C13_RG,consonant,"""M""",88.081,88.191,0.110,33
24341,C13_RG,consonant,"""Z""",88.191,88.281,0.090,33


**6. Save File Created by Going Through Participants Individually**

In [46]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline


In [47]:
dir = "2.BreathGroups_Assigned"

if os.path.exists(dir) == False:
  os.mkdir(dir)

In [111]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned/

/content/gdrive/MyDrive/RhythmAnalysisPipeline/2.BreathGroups_Assigned


In [112]:
read_TextGrid_duration.to_excel("reading_TextGrid_comb_BG.xlsx")

In [113]:
%cd /content/gdrive/MyDrive/RhythmAnalysisPipeline/

/content/gdrive/MyDrive/RhythmAnalysisPipeline
